<a href="https://colab.research.google.com/github/SokKanaTorajd/nlp-practices/blob/main/tm_Tensor_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip uninstall tensorflow
# !pip install tensorflow-gpu==2.0.0-alpha0

In [ ]:
import tensorflow as tf

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import operator
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers

%matplotlib inline
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
print(tf.__version__)

2.6.0


In [ ]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

print("Device name: {}".format((x.device)))

Is there a GPU available: 
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
Is the Tensor on GPU #0:  
True
Device name: /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/") 

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd gdrive/MyDrive/Colab\ Notebooks/data

/content/gdrive/MyDrive/Colab Notebooks/data


In [ ]:
import pandas as pd
file_dir = '/content/drive/MyDrive/Colab Notebooks/labelled-sentiment-copy.csv'
df  = pd.read_csv(file_dir)
# df_test  = pd.read_csv("Unlabeled.csv")

FileNotFoundError: ignored

In [ ]:
X_train, X_test  = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
y_train, y_test = X_train['sentimen'].values, X_test['sentimen'].values

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

In [ ]:
X_train = X_train['Tweet'].fillna('_NA_').values
X_test = X_test['Tweet'].fillna('_NA_').values
# X_submission = df_test['Tweet'].fillna('_NA_').values

In [ ]:
X_train.shape

(9725,)

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
# X_submission = tokenizer.texts_to_sequences(X_submission)

In [ ]:
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
# X_submission = pad_sequences(X_submission, maxlen=maxlen)

In [ ]:
# def data_prep(df):
#     print("Splitting dataframe with shape {} into training and test datasets".format(df.shape))
#     X_train, X_test  = train_test_split(df, test_size=0.1, random_state=2019)
#     y_train, y_test = X_train['target'].values, X_test['target'].values
    
#     print("Filling missing values")
#     X_train = X_train['Tweet'].fillna('_NA_').values
#     X_test = X_test['Tweet'].fillna('_NA_').values
#     X_submission = df_test['Tweet'].fillna('_NA_').values
    
#     print("Tokenizing {} questions into words".format(df.shape[0]))
#     tokenizer = Tokenizer(num_words=max_features)
#     tokenizer.fit_on_texts(list(X_train))
#     X_train = tokenizer.texts_to_sequences(X_train)
#     X_test = tokenizer.texts_to_sequences(X_test)
#     X_submission = tokenizer.texts_to_sequences(X_submission)
    
#     print("Padding sequences for uniform dimensions")
#     X_train = pad_sequences(X_train, maxlen=maxlen)
#     X_test = pad_sequences(X_test, maxlen=maxlen)
#     X_submission = pad_sequences(X_submission, maxlen=maxlen)
    
#     print("Completed data preparation, returning training, test and submission datasets, split as dependent(X) and independent(Y) variables")
    
#     return X_train, X_test, y_train, y_test, X_submission

In [ ]:
# X_train, X_test, y_train, y_test, X_submission = data_prep(df)

In [ ]:
model1 = Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen))
model1.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model1.add(Dropout(0.2))
# model1.add(GlobalMaxPool1D())
model1.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(CuDNNLSTM(32))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', 
               metrics=['accuracy'])

model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 100, 256)          440320    
_________________________________________________________________
dropout_15 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 100, 128)          164864    
_________________________________________________________________
dropout_16 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
cu_dnnlstm_16 (CuDNNLSTM)    (None, 32)                20736     
_________________________________________________________________
dense_12 (Dense)             (None, 64)               

In [ ]:
%time model1.fit(X_train, y_train, batch_size=512, epochs=35, validation_data=(X_test, y_test), verbose = 1)

Epoch 1/35
19/19 [==============================] - 9s 302ms/step - loss: 0.2099 - accuracy: 0.4905 - val_loss: -0.1764 - val_accuracy: 0.5032
Epoch 2/35
19/19 [==============================] - 5s 256ms/step - loss: -0.1593 - accuracy: 0.4918 - val_loss: -0.4110 - val_accuracy: 0.5032
Epoch 3/35
19/19 [==============================] - 5s 255ms/step - loss: -0.3049 - accuracy: 0.4918 - val_loss: -0.6718 - val_accuracy: 0.5032
Epoch 4/35
19/19 [==============================] - 5s 255ms/step - loss: -0.4635 - accuracy: 0.4918 - val_loss: -0.9919 - val_accuracy: 0.5032
Epoch 5/35
19/19 [==============================] - 5s 255ms/step - loss: -0.6994 - accuracy: 0.4918 - val_loss: -1.5664 - val_accuracy: 0.5032
Epoch 6/35
19/19 [==============================] - 5s 253ms/step - loss: -1.1093 - accuracy: 0.4918 - val_loss: -2.1401 - val_accuracy: 0.5032
Epoch 7/35
19/19 [==============================] - 5s 252ms/step - loss: -1.4291 - accuracy: 0.4918 - val_loss: -3.1776 - val_accuracy: 

In [ ]:
import numpy as np
embedding_matrix = np.zeros((1000 + 1, 16))


In [ ]:
embedding_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])